In [1]:
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
import scipy.misc
import tensorflow as tf

In [2]:
# 创建图会话
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)
# 导入模型， .pb文件
model_fn = 'tensorflow_inception_graph.pb' # 导入inception网络

# 这里下载的是 V1 , V3的数据结构不太一样代码也不一样哈
# InceptionV1 model (52MB):
# https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip

# InceptionV3 model(90MB):
# https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz
# 这里略过下载+解压代码 直接导入
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    # 读取二进制文件f
    graph_def.ParseFromString(f.read())

In [3]:
# 定义输入图像的占位符
t_input = tf.placeholder(np.float32, name='input')

# 图像预处理值 --- 减去像素均值
imagenet_mean = 117.0  # 在训练inception模型时进行了减均值处理，此处也需要减同样的均值以保持一致

# 图像预处理 --- 增加维度
# 图像格式 (height, width, channels) 为同时将多张图片输入网络，变为(batch, height, width, channels)
t_preprocessed = tf.expand_dims(t_input - imagenet_mean, 0)

print(t_preprocessed)
# 导入模型并将预处理的图像送入网络中
tf.import_graph_def(graph_def,input_map={'input': t_preprocessed})

Tensor("ExpandDims:0", dtype=float32)


In [4]:
# 找出卷积层 graph.get_operations() Conv2D意思是二维卷积
layers = [op.name for op in graph.get_operations() if op.type == 'Conv2D']
# 输出卷积层层数
print('Number of layers : ', len(layers))
# 所有卷积层名称
print(layers) # 59个卷积层
# 指定卷积层的参数

def printArg(name):
    print('shape of %s:%s' % (name1, str(graph.get_tensor_by_name('import/'+ name + ':0').get_shape())))
    
# 因为此时还不清楚输入图像的数量和大小，所以前三个维度是不确定的，显示？
# 由于导入的是已经训练好的模型，所以指定的卷积层通道数量是固定的
name1 = 'mixed4d_3x3_bottleneck_pre_relu'
printArg(name1)

name2 = 'mixed4e_5x5_bottleneck_pre_relu'
printArg(name2)

Number of layers :  59
['import/conv2d0_pre_relu/conv', 'import/conv2d1_pre_relu/conv', 'import/conv2d2_pre_relu/conv', 'import/mixed3a_1x1_pre_relu/conv', 'import/mixed3a_3x3_bottleneck_pre_relu/conv', 'import/mixed3a_3x3_pre_relu/conv', 'import/mixed3a_5x5_bottleneck_pre_relu/conv', 'import/mixed3a_5x5_pre_relu/conv', 'import/mixed3a_pool_reduce_pre_relu/conv', 'import/mixed3b_1x1_pre_relu/conv', 'import/mixed3b_3x3_bottleneck_pre_relu/conv', 'import/mixed3b_3x3_pre_relu/conv', 'import/mixed3b_5x5_bottleneck_pre_relu/conv', 'import/mixed3b_5x5_pre_relu/conv', 'import/mixed3b_pool_reduce_pre_relu/conv', 'import/mixed4a_1x1_pre_relu/conv', 'import/mixed4a_3x3_bottleneck_pre_relu/conv', 'import/mixed4a_3x3_pre_relu/conv', 'import/mixed4a_5x5_bottleneck_pre_relu/conv', 'import/mixed4a_5x5_pre_relu/conv', 'import/mixed4a_pool_reduce_pre_relu/conv', 'import/mixed4b_1x1_pre_relu/conv', 'import/mixed4b_3x3_bottleneck_pre_relu/conv', 'import/mixed4b_3x3_pre_relu/conv', 'import/mixed4b_5x5_bot

In [8]:
DEST_IMAGE = 'naive_deepdream.jpg'
SAVE_IMAGE = 'naive_single_chn.jpg'

# 定义一些图像操作函数
# 吧numpy.ndarray 保存成图像文件
def savearray(img_array,img_name):
    scipy.misc.toimage(img_array).save(img_name)
    print('img saved: %s' % img_name)
    
# 渲染函数
def render_naive(t_obj, img0, iter_n=20, step=1.0):
    # t_obj：是layer_output[:, :, :, channel]，即卷积层某个通道的值
    # img0：初始图像（噪声图像）
    # iter_n：迭代次数
    # step：用于控制每次迭代步长，可以看作学习率   
    
    t_score = tf.reduce_mean(t_obj)
    # t_score是t_obj的平均值
    # 由于我们的目标是调整输入图像使卷积层激活值尽可能大
    # 即最大化t_score
    # 为达到此目标，可使用梯度下降
    # 计算t_score对t_input的梯度
    t_grad = tf.gradients(t_score, t_input)[0]
    
    img = img0.copy() #复制新图像可避免影响原图像的值
    for i in range(iter_n):
        # 在sess 中计算梯度， 以及当前的t_score
        g, score = sess.run([t_grad, t_score], {t_input:img})
        # 对img应用梯度
        # 首先对梯度进行归一化处理
        g /= g.std() + 1e-8
        # 将正规化处理后的梯度应用在图像上，step用于控制每次迭代步长，此处为1.0
        img += g * step
        #print('score(mean)=%f' % (score))
        print('iter:%d' %(i+1), 'score(mean)=%f' % score)
    # 保存图片
    savearray(img, DEST_IMAGE)
    
    
# 可视化指定通道处理后的图像
def showImageAfterSingleChannel(name, channel):
    # layer_out[:,:,:,channel]即可表示第channel+1个通道即第140个通道
    layer_output = graph.get_tensor_by_name('import/%s:0' % name)
    
    # 定义图像噪声
    # inception 图像大小为 224 * 224 * 3
    img_noise = np.random.uniform(size=(224,224,3)) + 100.0
    
    # 调用render_naive 函数渲染并保存图片
    render_naive(layer_output[:,:,:,channel], img_noise, iter_n=20)
    
    # 显示图片
    im = PIL.Image.open(DEST_IMAGE)
    im.show()
    im.save(SAVE_IMAGE)
    

In [6]:
# 通过单通道特征生成DeepDream图像
# 定义卷积层，通道数，并取出对应的tensor 
# 4d表示第四层
name = 'mixed4d_3x3_bottleneck_pre_relu' # (?, ?, ?, 144)
# mixed4d_3x3_bottleneck_pre_relu总共144个通道可以选取0~143中任意整数进行最大化
channel = 139

showImageAfterSingleChannel(name, channel)


iter:1 score(mean)=-19.544886
iter:2 score(mean)=-27.833132
iter:3 score(mean)=29.033306
iter:4 score(mean)=92.798988
iter:5 score(mean)=154.567245
iter:6 score(mean)=228.642914
iter:7 score(mean)=273.431610
iter:8 score(mean)=323.912384
iter:9 score(mean)=380.070862
iter:10 score(mean)=424.626587
iter:11 score(mean)=481.944122
iter:12 score(mean)=518.043701
iter:13 score(mean)=551.315063
iter:14 score(mean)=581.729126
iter:15 score(mean)=608.972412
iter:16 score(mean)=644.366394
iter:17 score(mean)=667.989258
iter:18 score(mean)=697.467163
iter:19 score(mean)=712.710144
iter:20 score(mean)=740.852844
img saved: naive_deepdream.jpg


In [9]:
# 较低层单通道卷积特征生成的deepdream图像  第3层
name3 = 'mixed3a_3x3_bottleneck_pre_relu' # (?, ?, ?, 86)
# mixed4d_3x3_bottleneck_pre_relu总共144个通道可以选取0~143中任意整数进行最大化
channel = 86

showImageAfterSingleChannel(name3, channel)

iter:1 score(mean)=5.845768
iter:2 score(mean)=49.074760
iter:3 score(mean)=136.151733
iter:4 score(mean)=205.071991
iter:5 score(mean)=250.079361
iter:6 score(mean)=277.414886
iter:7 score(mean)=299.064667
iter:8 score(mean)=316.046600
iter:9 score(mean)=328.602112
iter:10 score(mean)=339.448090
iter:11 score(mean)=347.278992
iter:12 score(mean)=355.681641
iter:13 score(mean)=361.482208
iter:14 score(mean)=367.130035
iter:15 score(mean)=371.916290
iter:16 score(mean)=376.022400
iter:17 score(mean)=380.178345
iter:18 score(mean)=383.393097
iter:19 score(mean)=386.982666
iter:20 score(mean)=389.501129
img saved: naive_deepdream.jpg


In [11]:
# 较高层单通道卷积特征生成的deepdream图像  第5层
name4 = 'mixed5b_5x5_pre_relu' # (?, ?, ?, 128)
# mixed4d_3x3_bottleneck_pre_relu总共144个通道可以选取0~143中任意整数进行最大化
channel = 118

showImageAfterSingleChannel(name4, channel)

iter:1 score(mean)=-7.930314
iter:2 score(mean)=-8.057145
iter:3 score(mean)=-2.405256
iter:4 score(mean)=5.294808
iter:5 score(mean)=12.732895
iter:6 score(mean)=23.550644
iter:7 score(mean)=34.669746
iter:8 score(mean)=39.058891
iter:9 score(mean)=56.442463
iter:10 score(mean)=62.099545
iter:11 score(mean)=69.372742
iter:12 score(mean)=75.633667
iter:13 score(mean)=81.130478
iter:14 score(mean)=95.268074
iter:15 score(mean)=103.205635
iter:16 score(mean)=112.273117
iter:17 score(mean)=119.022392
iter:18 score(mean)=127.181740
iter:19 score(mean)=131.421310
iter:20 score(mean)=142.202713
img saved: naive_deepdream.jpg


In [12]:
# 用所有通道处理后的图像
def showImageAfterLayer(name):
    layer_output = graph.get_tensor_by_name('import/%s:0' % name)
    
    # 定义图像噪声
    # inception 图像大小为 224 * 224 * 3
    img_noise = np.random.uniform(size=(224,224,3)) + 100.0
    
    # 调用render_naive 函数渲染并保存图片
    # 这里之前第一个参数是layer_output[:,:,:,channel]
    render_naive(layer_output, img_noise, iter_n=20)
    
    # 显示图片
    im = PIL.Image.open(DEST_IMAGE)
    im.show()
    im.save('all_chn.jpg')
    
showImageAfterLayer(name)

iter:1 score(mean)=-6.872653
iter:2 score(mean)=-10.039657
iter:3 score(mean)=-6.592560
iter:4 score(mean)=-0.465422
iter:5 score(mean)=7.240916
iter:6 score(mean)=12.084120
iter:7 score(mean)=16.269520
iter:8 score(mean)=18.786577
iter:9 score(mean)=21.772913
iter:10 score(mean)=24.574852
iter:11 score(mean)=28.374317
iter:12 score(mean)=31.162733
iter:13 score(mean)=34.968575
iter:14 score(mean)=37.962578
iter:15 score(mean)=38.809959
iter:16 score(mean)=43.739983
iter:17 score(mean)=44.681900
iter:18 score(mean)=47.768284
iter:19 score(mean)=48.392441
iter:20 score(mean)=51.420448
img saved: naive_deepdream.jpg


In [16]:
# 以图像为起点生成Deep dream
def showImageUseBackground(name):
    layer_output = graph.get_tensor_by_name('import/%s:0' % name)
    
    # *****这里不再使用噪声
    # img_noise = np.random.uniform(size=(224,224,3)) + 100.0
    # 取而代之的是图像
    img_noise = PIL.Image.open('car.jpeg')
    
    # 调用render_naive 函数渲染并保存图片 我们把迭代周期调整到100
    render_naive(layer_output, img_noise, iter_n=100)
    
    # 显示图片
    im = PIL.Image.open(DEST_IMAGE)
    im.show()
    im.save('all_chn.jpg')

# 使用第四层处理 (?, ?, ?, 512)
showImageUseBackground('mixed4c')

iter:1 score(mean)=18.461956
iter:2 score(mean)=23.401796
iter:3 score(mean)=28.657604
iter:4 score(mean)=33.663586
iter:5 score(mean)=37.859127
iter:6 score(mean)=41.594383
iter:7 score(mean)=44.747818
iter:8 score(mean)=47.840309
iter:9 score(mean)=50.579010
iter:10 score(mean)=53.081329
iter:11 score(mean)=55.680794
iter:12 score(mean)=57.924217
iter:13 score(mean)=59.972481
iter:14 score(mean)=61.895298
iter:15 score(mean)=63.755913
iter:16 score(mean)=65.416740
iter:17 score(mean)=66.950172
iter:18 score(mean)=68.482933
iter:19 score(mean)=69.975571
iter:20 score(mean)=71.425934
iter:21 score(mean)=72.671272
iter:22 score(mean)=73.860558
iter:23 score(mean)=75.014801
iter:24 score(mean)=76.076790
iter:25 score(mean)=77.371918
iter:26 score(mean)=78.065933
iter:27 score(mean)=79.336113
iter:28 score(mean)=80.096436
iter:29 score(mean)=81.198555
iter:30 score(mean)=81.884560
iter:31 score(mean)=82.897751
iter:32 score(mean)=83.601372
iter:33 score(mean)=84.460770
iter:34 score(mean)